## Frames of Reference:
* *I*: Celestial/Inertial frame, such as GCI, is likely result of Star Tracker calculations.
* *E*: ECEF (geocentric) or a geodetic model, such as WGS-84, is likely result of GPS calculations.
* *O*: LVLH or orbital frame is relevant from Control perspective.
* *B*: Satellite body frame is determined by satellite geometry and defined here as the principle axes.

All of these reference frames are likely to be relevant, so we must provide transformations between them.

Any transformation to the body frame is simply a matter of determining the orientation, i.e. attitude, of the body frame in whatever other frame is referenced.

The transformation from GCI to ECEF is given by a rotation angle, Greenwich Mean Sidereal Time angle, around their shared z-axis (the North Pole). Thus, it is a time dependent relation.

The transformation from LVLH to GCI is given by the spacecraft position and velocity in the inertial frame. Thus it is a time (and orbit) dependent relation.

Please note that here we are considering transformations in the *alias* sense (rather than the *alibi* sense), as rotations of reference frames around fixed abstract vectors.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# see pages 32-34 of Fundamentals of Spacecraft Attitude
# Determination and Control, Markely for reasoning here
def julian_date(Y, M, D, h, m, s, leap=False):
    sixty = 60 if not leap else 61
    
    part1 = int(7/4 * (Y + int((M + 9) / 12)))
    part2 = int(275*M / 9)
    part3 = (60*h + m + s/sixty) / 1440
    
    return 1721013.5 + 367*Y + D - part1 + part2 + part3

# from epoch J2000.0 to zero hour now
def centuries_elapsed(Y, M, D, leap=False):
    return (julian_date(Y, M, D,0,0,0,leap) - 2451545) / 36525

def gmst_s(Y, M, D, h, m, s, leap=False):
    t0 = centuries_elapsed(Y, M, D, leap)
    return 24110.54841 + 8640184.812866 * t0 + \
        0.093104 * t0**2 - 6.2*10**(-6) * t0**3 + \
        1.002737909350795 * (3600 * h + 60 * m + s)

# Greenwich Mean Sidereal Time angle
def theta_gmst(Y, M, D, h, m, s, leap=False):
    gmst = gmst_s(Y, M, D, h, m, s, leap) % 86400
    return np.radians(gmst / 240)

# logical implementation of gregor's calendar, we should track leap seconds somehow?
def time_update(Y, M, D, h, m, s, leap):
    def leap_year(Y):
        if not Y % 4 == 0:
            return False
        elif not Y % 100 == 0:
            return True
        elif not Y % 400 == 0:
            return False
        else:
            return True
    
    if (not leap and s == 60) or (leap and s == 61):
        s = 0
        leap = False
        m += 1
    if m == 60:
        m = 0
        h += 1
    if h == 24:
        h = 0
        D += 1
    if ((M in [1, 3, 5, 7, 8, 10, 12] and D == 31) or
        (M in [4, 6, 9, 11] and D == 30) or
        (M == 2 and ((leap_year(Y) and D == 29) or
                    D == 28))): # I HATE the Gregorian calendar
        D = 1
        M += 1
    if M == 12:
        M = 1
        Y += 1
    return Y, M, D, h, m, s, leap

In [3]:
# transformation from inertial frame to ECEF, Cartesian x, y, z
def transform_EI(Y, M, D, h, m, s, leap=False):
    gmst = theta_gmst(Y, M, D, h, m, s, leap)
    C = np.cos(gmst)
    S = np.sin(gmst)
    return np.array([[C, S, 0],
                     [-S, C, 0],
                     [0, 0, 1]])

# transformation from orbital frame to inertial frame, Cartesian x, y, z
def transform_IO(r_I, v_I):
    o_3I = - r_I / np.linalg.norm(r_I)
    o_2I = - np.cross(r_I, v_I) / np.linalg.norm(np.cross(r_I, v_I))
    o_1I = np.cross(o_2I, o_3I)
    return np.array([[o_1I[0], o_2I[0], o_3I[0]],
                     [o_1I[1], o_2I[1], o_3I[1]],
                     [o_1I[2], o_2I[2], o_3I[2]]])

## Star Tracker
The Star Tracker camera faces the +X direction of the satellite body frame and provides the Right Ascension (RA) and Declination (dec) of the center of images from the camera in terms of the Hipparcos star catalog from the ESA, as well as the orientation angle which is positive for a clockwise rotation about the camera axis and negative otherwise. This is an inertial frame for our solar system, those interested in details should check here https://aa.usno.navy.mil/faq/docs/ICRS_doc.php

For our purposes, the approximation of the Geocentric Inertial Frame should suffice. We must provide methods for converting our Star Tracker output to quaternions now.

The Star Tracker gives the attitude of the +X_B axis of our satellite in the inertial frame, we can think of this attitude in terms of Tait-Bryan angles and then convert it to the quaternion representation. This is helpful because by applying that transformation to Z_I instead of X_I, we will obtain the attitude of Z_B in the inertial frame.

In [4]:
# rotation quaternion, acts on reference frame, 3-2-1 order
def eulerangle_to_quat(RA, dec, orientation):
    c_phi = np.cos(np.radians(RA)/2)
    s_phi = np.sin(np.radians(RA)/2)
    c_theta = np.cos(np.radians(dec)/2)
    s_theta = np.sin(np.radians(dec)/2)
    c_psi = np.cos(np.radians(orientation)/2)
    s_psi = np.sin(np.radians(orientation)/2)
    return np.array([c_phi * c_theta * c_psi + s_phi * s_theta * s_psi,
                     c_phi * c_theta * s_psi + s_phi * s_theta * c_psi,
                     c_phi * s_theta * c_psi + s_phi * c_theta * s_psi,
                     s_phi * c_theta * c_psi + c_phi * s_theta * s_psi])
    

## Attitude Representations
Most of the above transformations are given in the form of Direction Cosine Matrices, which act on vectors in a given Cartesian reference frame to provide their representations in another such frame. Other types of representation we will not consider here are Euler angles and Rodrigues parameters. However, we will consider unit quaternions as a convenient representation of Euler axis/angle pairs.

As such, we will provide methods for translating between quaternions and DCMs.

I got this from https://www.tu-berlin.de/fileadmin/fg169/miscellaneous/Quaternions.pdf

Please note that we are defining quaternions as such, $$q = [s, \vec{v}^T]^T = [q_0, q_1, q_2, q_3]^T$$
where $q_0$ is the scalar part and $q_1, q_2, q_3$ are the vector part.

In [5]:
# this could be rewritten as a function of the next function but i'm lazy
def rotationvector_to_quat(r):
    nr = np.linalg.norm(r)
    v = np.sin(nr/2) / nr * r
    return np.array([np.cos(nr/2), v[0], v[1], v[2]])

# this is the most straightforward way to understand the meaning of a unit quaterion
def axisangle_to_quat(r, theta):
    v = np.sin(theta/2) * r
    return np.array([np.cos(theta/2), v[0], v[1], v[2]])

# in case we want to decompose quats to something more intuitive
def quat_to_axisangle(q):
    norm = np.linalg.norm(q[1:])
    theta = 2 * np.arctan2(norm, q[0])
    if theta == 0:
        v = np.zeros(3)
    else:
        v = q[1:] / norm
    return v, theta

# there are a number of ways to do this, this is safest, numerically
def DCM_to_quat(mtrx):
    q0 = sqrt(1/4 * (1 + np.trace(mtrx)))
    q1 = sqrt(1/4 * (1 - np.trace(mtrx) + 2 * mtrx[0][0]))
    q2 = sqrt(1/4 * (1 - np.trace(mtrx) + 2 * mtrx[1][1]))
    q3 = sqrt(1/4 * (1 - np.trace(mtrx) + 2 * mtrx[2][2]))
    q = np.array([q0, q1, q2, q3])
    maxq = max(q)
    for i, qi in enumerate(q):
        if qi == maxq:
            index = i
            break
    if index == 0:
        q[1] = (mtrx[2][1] - mtrx[1][2]) / (4 * q[1])
        q[2] = (mtrx[0][2] - mtrx[2][0]) / (4 * q[2])
        q[3] = (mtrx[1][0] - mtrx[0][1]) / (4 * q[3])
    elif index == 1:
        q[0] = (mtrx[2][1] - mtrx[1][2]) / (4 * q[0])
        q[2] = (mtrx[1][0] + mtrx[0][1]) / (4 * q[2])
        q[3] = (mtrx[0][2] + mtrx[2][0]) / (4 * q[3])
    elif index == 2:
        q[0] = (mtrx[0][2] - mtrx[2][0]) / (4 * q[0])
        q[1] = (mtrx[1][0] + mtrx[0][1]) / (4 * q[1])
        q[3] = (mtrx[2][1] + mtrx[1][2]) / (4 * q[3])
    elif index == 3:
        q[0] = (mtrx[1][0] - mtrx[0][1]) / (4 * q[0])
        q[1] = (mtrx[0][2] + mtrx[2][0]) / (4 * q[1])
        q[2] = (mtrx[2][1] + mtrx[1][2]) / (4 * q[2])
    return q

# attitude matrix from quat
def quat_to_DCM(q):
    return np.array([[q[0]**2 + q[1]**2 - q[2]**2 - q[3]**2, 2*(q[1]*q[2] - q[3]*q[0]), 2*(q[1]*q[3] + q[2]*q[0])],
                     [2*(q[1]*q[2] + q[3]*q[0]), q[0]**2 - q[1]**2 + q[2]**2 - q[3]**2, 2*(q[2]*q[3] - q[1]*q[0])],
                     [2*(q[1]*q[3] - q[2]*q[0]), 2*(q[2]*q[3] + q[1]*q[0]), q[0]**2 - q[1]**2 - q[2]**2 + q[3]**2]])


In [6]:
def geodetic_to_ECEF(lat, long, h):
    a = 6378137.0 # m, semimajor axis
    e = 0.0818 # eccentricity approximation
    N = a / np.sqrt(1 - (e*np.sin(lat))**2)
    temp = (N + h) * np.cos(lat)
    x = temp * np.cos(long)
    y = temp * np.sin(long)
    z = (N*(1 - e**2) + h) * np.sin(lat)
    return np.array([x, y, z])

# world geodetic system 1984 model
def ECEF_to_geodetic(r):
    a = 6378137.0 # m, semimajor axis
    b = 6356752.3142 # m, semiminor axis
    x = r[0]*1000 # convert from km to m
    y = r[1]*1000
    z = r[2]*1000
    # convert from ECEF to geodetic coords
    # see page 35 and pray there are no typos in the book
    def geo_rho(x, y):
        return np.linalg.norm(np.array([x, y]))

    def geo_e(a, b):
        return np.sqrt(1 - (b/a)**2)

    def geo_eps(a, b):
        return np.sqrt((a/b)**2 - 1)

    def geo_p(z, eps):
        return abs(z) / eps**2

    def geo_s(rho, e, eps):
        return (rho / (e*eps))**2

    def geo_q(p, b, s):
        return p**2 - b**2 + s

    def geo_u(p, q):
        return p / np.sqrt(q)

    def geo_v(b, u, q):
        return (b*u)**2 / q

    def geo_P(v, s, q):
        return 27*v*s/q

    def geo_Q(P):
        return (np.sqrt(P+1) + np.sqrt(P))**(2/3)

    def geo_t(Q):
        return (1 + Q + 1/Q)/6

    def geo_c(u, t):
        return np.sqrt(u**2 - 1 + 2*t)

    def geo_w(c, u):
        return (c - u)/2

    def geo_d(z, q, u, v, w, t):
        return np.sign(z)*np.sqrt(q)*(w +
                                     np.sqrt(
                                     np.sqrt(t**2 + v) -
                                     u * w - t/2 - 1/4))

    def geo_N(a, eps, d, b):
        return a * np.sqrt(1+ (eps*d/b)**2)
    
    # latitude
    def geo_lam(eps, d, N):
        return np.arcsin((eps**2 + 1)*d/N)

    # longitude
    def geo_h(rho, z, a, N, lam): # height above geode
        return rho*np.cos(lam) + z*np.sin(lam) - a**2 / N
    
    # height
    def geo_phi(x, y):
        return np.arctan2(y, x)
    
    e = geo_e(a, b)
    eps = geo_eps(a, b)
    rho = geo_rho(x, y)
    p = geo_p(z, eps)
    s = geo_s(rho, e, eps)
    q = geo_q(p, b, s)
    u = geo_u(p, q)
    v = geo_v(b, u, q)
    P = geo_P(v, s, q)
    Q = geo_Q(P)
    t = geo_t(Q)
    c = geo_c(u, t)
    w = geo_w(c, u)
    d = geo_d(z, q, u, v, w, t)
    N = geo_N(a, eps, d, b)
    
    lam = geo_lam(eps, d, N)
    h = geo_h(rho, z, a, N, lam)
    phi = geo_phi(x, y)
    return np.array([lam, phi, h]) # lat, long, height

In [7]:
# this is the matrix corresponding to a vector acting by cross product
def cross(w):
    return np.array([[0, -w[2], w[1]],
                     [w[2], 0, -w[0]],
                     [-w[1], w[0], 0]])

# this is hamilton's quaternion product
def product(a, b):
    v = b[0] * a[1:] + a[0] * b[1:] + np.cross(a[1:], b[1:])
    return np.array([a[0] * b[0] - np.dot(a[1:], b[1:]), v[0], v[1], v[2]])

# this is used by some textbooks, i don't use it personally
def compose(a, b):
    return product(b, a)

# this is the inverse of a unit quat
def conjugate(q):
    return np.array([q[0], -q[1], -q[2], -q[3]])

# this rotates a vector with a fixed frame
def conjugation(q, v):
    return product(q, product(np.array([0, v[0], v[1], v[2]]), conjugate(q)))[1:]

# this rotates a frame with a fixed vector
def frame_conjugation(q, v):
    return conjugation(conjugate(q), v)

# this constrains a quat to S3
def renormalize(q):
    return q / np.linalg.norm(q)